In [11]:
import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from glob import glob
import os
import xml.etree.ElementTree as et
import numpy as np
import pandas as pd

In [3]:
set_number = '5'
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk"
verification_dir_set5 = os.path.join(parent_directory,"unverified_images/student_reviewed_unverified_images_set7")
#verification_dir = os.path.join(parent_directory,"verification_set"+set_number)
#folder_names = ["Niculescu_Sunny_Poonacha"]                            
#annotator_allocation = ['Mia','Tang']
#tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'

In [4]:
os.listdir(verification_dir_set5)

['Cleave', 'Niculescu', 'Poonacha', 'Sunny']

## Determine the number of images labeled by each annotator for the verification seperation

In [7]:
img_anno=ap.img_path_anno_path(ap.list_of_sub_directories(verification_dir_set5))

In [16]:
annotators = []
num_images_annotated = []

#the total number of images in each allocated set for each annotator
for i in range(len(img_anno)):
    #print("Annotator:", img_anno[i,1].rsplit("/")[5],"; images:", len(glob(img_anno[i,1]+'/*')))
    annotators.append(img_anno[i,1].rsplit("/")[-3])
    num_images_annotated.append(len(glob(img_anno[i,1]+'/*')))
    fc.remove_thumbs(img_anno[i,0])
annotated_images_by_allocation = np.array((annotators, num_images_annotated)).T

#the total number of images by each annotator
unique_annotators = []
total_num_images_annotated = []
for i in np.unique(annotators):
    unique_annotators.append(i)
    total_num_images_annotated.append(sum(annotated_images_by_allocation[annotated_images_by_allocation[:,0] == i][:,1].astype(int))) #determine the total number of images for each annotator 
total_num_of_images_by_annotator = np.array((unique_annotators, total_num_images_annotated)).T
print(total_num_of_images_by_annotator)

[['Cleave' '960']
 ['Niculescu' '139']
 ['Poonacha' '1308']
 ['Sunny' '134']]


In [20]:
annotated_images_by_allocation

array([['Cleave', '405'],
       ['Cleave', '528'],
       ['Desbans', '153'],
       ['Niculescu', '190'],
       ['Poonacha', '121'],
       ['Poonacha', '93'],
       ['Poonacha', '97'],
       ['Poonacha', '204']], dtype='<U11')

In [13]:
#print(Poonacha)
#total_num_of_images_by_annotator[0] #determine the total number of images for each annotator 
num_images1 = 0
verify_set_1 = ["Poonacha_Niculescu_Sunny"]
annotator_allocation_set_1 = ["Cleave", "Kang"]

num_images2 = 0
verify_set_2 = ["Cleave_Sunny_Poonacha"]
annotator_allocation_set_2 = ["Niculescu", "Alvarez", "Katpally", "Nayak", "Tang",'Celine']

num_images3 = 0
verify_set_3 = ["Poonacha_Niculescu_Cleave"] #went to Niculescu_Poonacha_Cleave
annotator_allocation_set_3 = ["Sunny","Jaewon", "Josh",'Feinberg']

num_images4 = 0
verify_set_4 = ["Niculescu_Cleave_Sunny"]
annotator_allocation_set_4 = ["Poonacha", "Shen"]

for i in range(len(total_num_of_images_by_annotator)):
    
    if total_num_of_images_by_annotator[i,0] in annotator_allocation_set_1:
        num_images1 += total_num_of_images_by_annotator[i,1].astype(int)

    if total_num_of_images_by_annotator[i,0] in annotator_allocation_set_2:
        num_images2 += total_num_of_images_by_annotator[i,1].astype(int)

    if total_num_of_images_by_annotator[i,0] in annotator_allocation_set_3:
        num_images3 += total_num_of_images_by_annotator[i,1].astype(int)
        
    if total_num_of_images_by_annotator[i,0] in annotator_allocation_set_4:
        num_images4 += total_num_of_images_by_annotator[i,1].astype(int)
        
print(verify_set_1, num_images1, annotator_allocation_set_1)
print(verify_set_2, num_images2, annotator_allocation_set_2)
print(verify_set_3, num_images3, annotator_allocation_set_3)
print(verify_set_4, num_images4, annotator_allocation_set_4)

['Poonacha_Niculescu_Sunny'] 1191 ['Cleave', 'Kang']
['Cleave_Sunny_Poonacha'] 985 ['Niculescu', 'Alvarez', 'Katpally', 'Nayak', 'Tang', 'Celine']
['Poonacha_Niculescu_Cleave'] 1140 ['Sunny', 'Jaewon', 'Josh', 'Feinberg']
['Niculescu_Cleave_Sunny'] 1104 ['Poonacha', 'Shen']


## test tracking array

In [14]:
img_anno = ap.img_path_anno_path(ap.list_of_sub_directories(verification_dir))

In [11]:
#tracking_array = ap.reference_image_annotation_file_with_annotator(img_anno, tracker_file_path) #load existing and update 

In [15]:
if os.path.exists(tracker_file_path):
    verfication_set = os.path.join(parent_directory, "verification_set" + set_number, "unverified_images")
    tile_img_annotation_annotator = ap.update_path(verfication_set, tracker_file_path)

folder_annotator_list, verification_dir = ap.verification_folders(parent_directory, verify_set_1, annotator_allocation_set_1, set_number)
tile_img_annotation_annotator = ap.seperate_images_for_verification_update_tracking(folder_annotator_list, verification_dir, 
                                                                                    set_number, tile_img_annotation_annotator)

Poonacha_Niculescu_Sunny
folder Poonacha_Niculescu_Sunny
annotator Cleave
annotator Kang
1191


In [16]:
np.save('outputs/tile_img_annotation_annotator.npy', tile_img_annotation_annotator)

column_names = ["tile_name", "chip_name", "chip pathway", "xml annotation", 
                "annotator - draw","annotator - verify coverage",
                "annotator - verify quality", "annotator - verify classes"]
tile_img_annotation_annotator_df = pd.DataFrame(data = tile_img_annotation_annotator, 
                                               index = tile_img_annotation_annotator[:,1], 
                                               columns = column_names)
tile_img_annotation_annotator_df.to_csv('outputs/tile_img_annotation_annotator_df.csv')

## Issues with correct inncorrect labels scipt
identify the issue with the annotation and correct in labelImg if needed

In [18]:
w = 15505 #the last annotation to be converted + 1

print(dist.chips_positive_list[w])
print(len(dist.chips_positive_list))
print(dist.chips_xml_list[w])
print(len(dist.chips_xml_list))

xml_file = dist.chips_xml_list[w]
print(xml_file)
# use the parse() function to load and parse an XML file
tree = et.parse(os.path.join(dist.chips_xml_dir, xml_file))
print(tree)
root = tree.getroot() 
print(root)
for obj in root.iter('object'):
    print(obj.text)
    for name in obj.findall('name'):
        print(name.text)

mn_60cm_2019_47096_m_4709624_nw_14_060_20190723_000203.xml
<Element 'annotation' at 0x0000013A0BF0CE00>

		
narrow_closed_roof_tank

		
narrow_closed_roof_tank

		
narrow_closed_roof_tank

		
narrow_closed_roof_tank

		
narrow_closed_roof_tank

		
narrow_closed_roof_tank

		
undefined_object

		
undefined_object

		
undefined_object

		
undefined_object

		
undefined_object

		
'


In [ ]:
positive_xml = os.listdir("//oit-nas-fe13dc.oit.duke.edu/data_commons-borsuk/complete_dataset/chips_positive_xml")
for i in range(len(positive_xml)):
    positive_xml[i] = positive_xml[i].split(".")[0]
    
positive = os.listdir("//oit-nas-fe13dc.oit.duke.edu/data_commons-borsuk/complete_dataset/chips_positive")
for i in range(len(positive)):
    positive[i] = positive[i].split(".")[0]
main_list = list(set(positive_xml) - set(positive))

positive = os.listdir("//oit-nas-fe13dc.oit.duke.edu/data_commons-borsuk/complete_dataset/chips_positive")
positive_recorded = list(np.load(tracker_file_path)[:,1])
main_list = list(set(positive_recorded) - set(positive))
len(main_list)

In [ ]:
#set3
#folder_names_set3 = ["anyone",  ]       # Tang#2         Poonacha#2   Sunny#2                         
#annotator_allocation_set3 = [['Josh','Jaewon',"Shen",'Katpally']]

In [1]:
import data_eng.az_proc as ap
import os
import numpy as np
import pandas as pd

In [ ]:
Make Corrections to Verification/Tracking 

## Correcting Verifiers in Tracking Numpy/DF

In [ ]:
#load tracking numpy
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

In [34]:
#specify the verifier set to correct and specify the correction that should be made
target_verifiers_1 = np.array(['Jaewon', 'James', 'Amadu'])
correction_1 = ['Jaewon', 'James', 'Poonacha']

target_verifiers_2 = np.array(['Jaewon', 'James', 'Josh'] )
correction_2 = ['Jaewon', 'Poonacha', 'Cleave']

target_verifiers_3 = np.array(['Josh', 'James','Amadu'])
correction_3 = ['Josh', 'Poonacha','Sunny']

target_verifiers_4 = np.array(['Cleave','Alvarez','Tang'])
correction_4 = ['Cleave','Tang','Poonacha']

target_verifiers_5 = np.array(['Sunny','Cleave','Alvarez'])
correction_5 = ['Sunny','Cleave','Poonacha']

In [35]:
#make corrections
for i in range(len(tile_img_annotation_annotator)):
    if np.array_equal(tile_img_annotation_annotator[i,5:8], target_verifiers_4):
        tile_img_annotation_annotator[i,5:8] = correction_4
    elif np.array_equal(tile_img_annotation_annotator[i,5:8], target_verifiers_5):
        tile_img_annotation_annotator[i,5:8] = correction_5

In [36]:
#save updated 
np.save('outputs/tile_img_annotation_annotator.npy', tile_img_annotation_annotator)
column_names = ["tile_name", "chip_name", "chip pathway", "xml annotation", 
                    "annotator - draw","annotator - verify coverage",
                    "annotator - verify quality", "annotator - verify classes"]
tile_img_annotation_annotator_df = pd.DataFrame(data = tile_img_annotation_annotator, 
                                                index = tile_img_annotation_annotator[:,1], 
                                                columns = column_names)
tile_img_annotation_annotator_df.to_csv('outputs/tile_img_annotation_annotator_df.csv')

## Count the Number of Images/Annotatiosn

In [56]:
files_verified = []
dirs =  ['D:/verification/verify_josh_james_amadu_1/chips',
                    'D:/verification/verify_jaewon_james_josh_1/chips',
                    'D:/verification/verify_jaewon_james_amadu_1/chips',
                    'D:/verification/verify_josh_jaewon_amadu_1/chips']
for dir in dirs:
    files_verified.append(os.listdir(dir))

files_verified = [ item for elem in files_verified for item in elem]
len(files_verified)
len(np.unique(np.array(files_verified)))


11271

In [34]:
files_to_be_verified = []
for dir in ap.img_path_anno_path(ap.list_of_sub_directories('D:/verification/unverified_images/student_reviewed_unverified_images'))[:,0]:
    files_to_be_verified.append(os.listdir(dir))

files_to_be_verified = [ item for elem in files_to_be_verified for item in elem]
len(files_to_be_verified)
len(np.unique(np.array(files_to_be_verified)))

11510

In [49]:
for uvfile in files_to_be_verified:
    if files_verified.count(uvfile) == 0:
        print(uvfile)
        
#Identify the position of missing files
items = [x for x in files_to_be_verified if 'Thumbs.db' in x]
indexes = [n for n, x in enumerate(files_to_be_verified) if'Thumbs.db' in x]

print(indexes)

Thumbs.db
Thumbs.db


In [47]:
#Identify Missing Files
missing = set(files_verified) - set(files_to_be_verified) 
missing

set()

In [3]:
ap.update_path(verification_dir, tracker_file_path)

array([['ny_60cm_2019_41073_m_4107341_sw_18_060_20190917',
        'ny_60cm_2019_41073_m_4107341_sw_18_060_20190917_000166.jpg',
        'D:\\Unverified_images\\student_reviewed_unverified_images/Aidan/unverified_images_not_reviewed_by_student1_Aidan/chips_positive\\ny_60cm_2019_41073_m_4107341_sw_18_060_20190917_000166.jpg',
        ..., 'Jaewon', 'Poonacha', 'Cleave'],
       ['ny_60cm_2019_41073_m_4107341_sw_18_060_20190917',
        'ny_60cm_2019_41073_m_4107341_sw_18_060_20190917_000342.jpg',
        'D:\\Unverified_images\\student_reviewed_unverified_images/Aidan/unverified_images_not_reviewed_by_student1_Aidan/chips_positive\\ny_60cm_2019_41073_m_4107341_sw_18_060_20190917_000342.jpg',
        ..., 'Jaewon', 'Poonacha', 'Cleave'],
       ['ny_60cm_2019_41073_m_4107341_sw_18_060_20190917',
        'ny_60cm_2019_41073_m_4107341_sw_18_060_20190917_000389.jpg',
        'D:\\Unverified_images\\student_reviewed_unverified_images/Aidan/unverified_images_not_reviewed_by_student1_Aidan/c